<a href="https://colab.research.google.com/github/anespart1/work_automation_use/blob/main/Labels_to_contents_20220320.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#-*- coding:UTF-8 -*-
import pandas as pd
import os, re, time
from google.colab import files

In [ ]:
#엑셀파일을 인식해서 분류해주는 기능
def load_excel_file():
  target, data = [], []
  for i in os.listdir(os.getcwd()):
    if '.xlsx' in i:
      if '코로나19' in i:
        data.append(i)
      else:
        target.append(i)
  return target, data

In [ ]:
target, data_list = load_excel_file()

In [ ]:
print(target)
print(data_list)

['NewsResult_20191201-20211130.xlsx']
['코로나19 팬데믹 이후 ‘진화’ 관련 뉴스기사 (경향신문2).xlsx', '코로나19 팬데믹 이후 ‘진화’ 관련 뉴스기사 (중앙일보1).xlsx', '코로나19 팬데믹 이후 ‘진화’ 관련 뉴스기사 (동아일보1).xlsx', '코로나19 팬데믹 이후 ‘진화’ 관련 뉴스기사 (조선일보2).xlsx', '코로나19 팬데믹 이후 ‘진화’ 관련 뉴스기사 (중앙일보2).xlsx', '코로나19 팬데믹 이후 ‘진화’ 관련 뉴스기사 (조선일보1).xlsx', '코로나19 팬데믹 이후 ‘진화’ 관련 뉴스기사 (경향신문1).xlsx', '코로나19 팬데믹 이후 ‘진화’ 관련 뉴스기사 (서울신문1).xlsx', '코로나19 팬데믹 이후 ‘진화’ 관련 뉴스기사 (서울신문2).xlsx', '코로나19 팬데믹 이후 ‘진화’ 관련 뉴스기사 (국민일보1).xlsx', '코로나19 팬데믹 이후 ‘진화’ 관련 뉴스기사 (문화일보1).xlsx', '코로나19 팬데믹 이후 ‘진화’ 관련 뉴스기사 (한겨레 신문1).xlsx', '코로나19 팬데믹 이후 ‘진화’ 관련 뉴스기사 (경향신문3).xlsx', '코로나19 팬데믹 이후 ‘진화’ 관련 뉴스기사 (동아일보2).xlsx', '코로나19 팬데믹 이후 ‘진화’ 관련 뉴스기사 (한국일보1).xlsx', '코로나19 팬데믹 이후 ‘진화’ 관련 뉴스기사 (조선일보3).xlsx', '코로나19 팬데믹 이후 ‘진화’ 관련 뉴스기사 (세계일보1).xlsx', '코로나19 팬데믹 이후 ‘진화’ 관련 뉴스기사 (한국일보2).xlsx', '코로나19 팬데믹 이후 ‘진화’ 관련 뉴스기사 (내일신문).xlsx']


In [ ]:
#최종 타깃 데이터 불러오기
target_df = pd.read_excel(target[0])
target_df.head()

,뉴스 식별자,일자,언론사,제목,본문,URL,분석제외 여부
0,1.100101e+06,20211017,경향신문,"NASA, 목성 주변 탐사선 ‘루시’ 발사 지구 생명체 탄생 비밀 찾는다",NaN,https://www.khan.co.kr/world/world-general/art...,NaN
1,1.100101e+06,20210721,경향신문,MZ세대가 말하는 ‘코로나 시대의 여행’ “랜선여행은 일시적 해소 방안일 뿐”,NaN,https://www.khan.co.kr/travel/national/article...,NaN
2,1.100101e+06,20211031,경향신문,[찌릿찌릿(知it智it) 전기 교실]휴대전화와 통신산업처럼 자동차와 에너지 산업 ‘...,NaN,https://www.khan.co.kr/science/science-general...,NaN
3,1.100101e+06,20211022,경향신문,3단 엔진 왜 꺼졌나 발사체 무게 형태 어떻게 진화시킬까,NaN,https://www.khan.co.kr/science/aerospace/artic...,NaN
4,1.100101e+06,20210704,경향신문,90광년 떨어진 파란 별 지구를 대신할 수 있을까,NaN,https://www.khan.co.kr/science/aerospace/artic...,NaN


In [ ]:
#데이터 항목별 LABEL 하나로 합치는 기능
def make_contents(dataframe=None):
  total_labels = []
  for i in range(len(dataframe)):
    labels = []
    for j in range(2, len(dataframe.columns)):          #LABEL이 몇개든 하나로
      if type(dataframe[dataframe.columns[j]][i]) is str:      #nan 값은 float라 에러를 내므로 걸러내야 함
        labels.append(dataframe[dataframe.columns[j]][i])
    labels = ' '.join(labels)                    #LABELS를 하나로 합치기
    total_labels.append(re.sub('  ', ' ', labels)) #띄어쓰기 2개인 경우를 1개로
  return total_labels

In [ ]:
#합쳐진 LABEL로 데이터프레임 새로 내보내는 기능
def merge_labels(dataframe=None):
  total_labels = make_contents(dataframe)
  ndf = dataframe[dataframe.columns[:2]].copy()
  ndf['TOT_LABELS'] = total_labels
  return ndf

In [ ]:
#뉴스별로 통합하고, 통합된 기사에 맞춰 데이터레이블 새로 내보내는 기능
def dissolve_contents(dataframe=None):
  #전역변수
  datakeys, page_urls, total_contents, content = [], [], [] ,[]
  key_checker = ''
  #조건반복
  for i in range(len(dataframe)):
    if i == 0:
      key_checker = dataframe[dataframe.columns[0]][i]
      datakeys.append(key_checker)
      page_urls.append(dataframe[dataframe.columns[1]][i])
    if key_checker != dataframe[dataframe.columns[0]][i]:
      key_checker = dataframe[dataframe.columns[0]][i]
      datakeys.append(key_checker)
      page_urls.append(dataframe[dataframe.columns[1]][i])
      total_contents.append(' '.join(content))
      content = []
    if type(dataframe[dataframe.columns[2]][i]) is str:
      content.append(dataframe[dataframe.columns[2]][i])
    if i == len(dataframe) -1:
      total_contents.append(' '.join(content))
  dissolved = pd.DataFrame({ "DATAKEY":datakeys, "PAGE_URL":page_urls, "CONTENT" :total_contents })
  return dissolved

In [ ]:
#data_list의 모든 엑셀파일을 1개로 합쳐주는 기능
def make_one_data_file(data_list=None):
  for num, loc in enumerate(data_list):
    df = pd.read_excel(loc)
    ndf = merge_labels(df)
    dissolved = dissolve_contents(ndf)
    if num == 0:
      excels = dissolved.copy()
    if num > 0:
      excels = pd.concat([excels, dissolved], ignore_index=True).copy()
  return excels

In [ ]:
excels = make_one_data_file(data_list)

In [ ]:
excels.head(5)

,DATAKEY,PAGE_URL,CONTENT
0,XRzau0PX,http://news.khan.co.kr/kh_news/khan_art_view.h...,SK텔레콤은 서울시내 교통 인프라를 5G로 연결하는 차세대 지능형 교통 시스템 ‘C...
1,iTqeI37y,http://news.khan.co.kr/kh_news/khan_art_view.h...,"여행이 일상에서 지워진 코로나19 시대, 사람들은 떠나지 못했다. 해외여행은 언감생..."
2,8apqgM7W,http://news.khan.co.kr/kh_news/khan_art_view.h...,효소는 세포 내외에서 일어나는 반응에서 촉매 역할을 하는 단백질이다. 화학촉매와 비...
3,O2cdMUy1,http://news.khan.co.kr/kh_news/khan_art_view.h...,10일 한국법제연구원(원장 김계홍)이 ‘디지털 전환 속의 Data 규제혁신: 산업계...
4,zfolJAz7,http://news.khan.co.kr/kh_news/khan_art_view.h...,편의점과 약국에서 해열진통제 ‘타이레놀’(성분명 아세트아미노펜)이 불티나게 팔리고 ...


In [ ]:
#NewsResult와 리스틀리 크롤링 결과 합치기
target_df = pd.merge(target_df, excels, how='left', left_on='URL', right_on='PAGE_URL')
target_df['제목+본문'] = target_df['제목'] + target_df['CONTENT'] #안종현 선생님 요청

In [ ]:
target_df.head(1)

,뉴스 식별자,일자,언론사,제목,본문,URL,분석제외 여부,DATAKEY,PAGE_URL,CONTENT,제목+본문
0,1.100101e+06,20211017,경향신문,"NASA, 목성 주변 탐사선 ‘루시’ 발사 지구 생명체 탄생 비밀 찾는다",NaN,https://www.khan.co.kr/world/world-general/art...,NaN,EnW5uiN5,https://www.khan.co.kr/world/world-general/art...,목성 궤도에 있는 소행성 탐사 임무를 최초로 맡은 우주선 루시호가 12년간의 대장정...,"NASA, 목성 주변 탐사선 ‘루시’ 발사 지구 생명체 탄생 비밀 찾는다목성 궤도..."


In [ ]:
#파일 생성, 다운로드, 제거
target_df.to_excel('processed_'+ target[0], sheet_name='new_name')

In [ ]:
#files.download('processed_'+ target[0])
#os.remove('processed_'+ target[0])